In [1]:
!pip install pyspark==3.0.0 koalas==1.0.1 pyarrow==0.15.1 scikit-learn==0.23.1

     |████████████████████████████████| 204.7MB 136kB/s eta 0:00:01   |█                               | 6.3MB 7.2MB/s eta 0:00:28     |█▊                              | 10.7MB 7.2MB/s eta 0:00:27     |███████████▏                    | 71.5MB 14.1MB/s eta 0:00:10     |█████████████████████████       | 160.6MB 9.0MB/s eta 0:00:05     |███████████████████████████████ | 198.6MB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 696kB 29.7MB/s eta 0:00:01
     |████████████████████████████████| 59.2MB 36.6MB/s eta 0:00:01
     |████████████████████████████████| 6.9MB 8.3MB/s eta 0:00:01
     |████████████████████████████████| 204kB 29.5MB/s eta 0:00:01
     |████████████████████████████████| 307kB 30.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.
ERROR: autoai-li

In [14]:
%%bash
for i in `seq 4`; do 
    rm log$i*
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log$i.txt
done

--2020-07-03 05:36:16--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19954 (19K) [text/plain]
Saving to: ‘log1.txt’

     0K .......... .........                                  100% 13.0M=0.001s

2020-07-03 05:36:16 (13.0 MB/s) - ‘log1.txt’ saved [19954/19954]

--2020-07-03 05:36:16--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/log2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13026 (13K) [text/plain]
Saving to: ‘log2.txt’

     0K .......... ..      

In [35]:
%%bash
for i in `seq 4`; do 
    echo "ts,rssi" > log$i.csv
    cat log$i.txt |awk  '{print $4","$11}' >> log$i.csv
done

In [17]:
%%bash
for i in `seq 4`; do 
    rm markers$i.csv
    wget https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers$i.csv
done

--2020-07-03 05:38:36--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers1.csv’

     0K                                                       100% 4.71M=0s

2020-07-03 05:38:36 (4.71 MB/s) - ‘markers1.csv’ saved [151/151]

--2020-07-03 05:38:36--  https://raw.githubusercontent.com/romeokienzler/BLE_Indoor_Location_Tracking/master/raw_data/markers2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 151 [text/plain]
Saving to: ‘markers2.csv’

     0K                        

In [19]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.getOrCreate()

In [38]:
from pyspark.sql.functions import lit

data = None
markers = None

for i in range(1,5):
    data_tmp = spark.read.option('header','true').csv('log'+str(i)+'.csv')
    data_tmp = data_tmp.withColumn('receiver_placement', lit(i))
    markers_tmp = spark.read.option('header','true').csv('markers'+str(i)+'.csv')
    markers_tmp = markers_tmp.withColumn('receiver_placement', lit(i))
    if i == 1:
        data = data_tmp
        markers = markers_tmp
    else:
        data.union(data_tmp)
        markers.union(markers_tmp)

In [39]:
#convert ts_string to ts_timestamp
from pyspark.sql.functions import to_timestamp
data = data.withColumn("ts_p", to_timestamp("ts", "HH:mm:ss"))
markers = markers.withColumn("ts_start_p", to_timestamp("ts_start", "HH:mm:ss"))
markers = markers.withColumn("ts_stop_p", to_timestamp("ts_stop", "HH:mm:ss"))

In [40]:
data.createOrReplaceTempView('data')
markers.createOrReplaceTempView('markers')

In [55]:
data = spark.sql('''
select rssi,d.receiver_placement,location from data d 
inner join markers m on 
d.receiver_placement = m.receiver_placement 
and d.ts_p>=m.ts_start_p 
and d.ts_p<=m.ts_stop_p
''')
data.createOrReplaceTempView('data')

In [46]:
import databricks.koalas as ks
kdf = data.to_koalas()

In [47]:
kdf = kdf.dropna()

In [48]:
pdf = kdf.toPandas()

In [56]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(pdf[['rssi','receiver_placement']], pdf[['location']])

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [57]:
from sklearn.metrics import accuracy_score
accuracy_score(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]))

0.7475728155339806

In [59]:
from sklearn.metrics import classification_report
target_names = ['0','1','2','3']
print(classification_report(pdf[['location']], clf.predict(pdf[['rssi','receiver_placement']]), target_names=target_names))

              precision    recall  f1-score   support

           0       0.92      0.88      0.90        41
           1       0.71      0.80      0.75        25
           2       0.00      0.00      0.00        15
           3       0.58      0.95      0.72        22

    accuracy                           0.75       103
   macro avg       0.56      0.66      0.59       103
weighted avg       0.67      0.75      0.70       103



/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
